In [3]:
import h5py
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error


In [4]:
# Load h5 data
with h5py.File('data\\train_df_30.h5', 'r') as f:
    y_train = f['y'][:]
with h5py.File('data\\validation_df_30.h5', 'r') as f:
    y_validation = f['y'][:]
with h5py.File('data\\rul_x_train_conv.h5', 'r') as f:
    x_train_rul = f['train'][:]
    x_validation_rul = f['validation'][:]
with h5py.File('data\\hs_x_train_conv.h5', 'r') as f:
    x_train_hs = f['train'][:]
    x_validation_hs = f['validation'][:]

In [6]:
# Separate RUL and Health State
y_rul_train = y_train[:, 0]
y_health_train = y_train[:, 1]

y_rul_validation = y_validation[:, 0]
y_health_validation = y_validation[:, 1]

In [ ]:
# Initialize CatBoost Classifier
hs_cb_model = CatBoostClassifier(
    depth=10,
    learning_rate=0.1,
    iterations=2000,
    loss_function='Logloss',
    eval_metric='Accuracy',
    verbose=1,
    random_state=42,
    early_stopping_rounds=50
)

# Train hs model
hs_cb_model.fit(x_train_hs, y_health_train, eval_set=(x_validation_hs, y_health_validation))

# Evaluate Model
y_pred_hs = hs_cb_model.predict(x_validation_hs)
hs_accuracy = accuracy_score(y_health_validation, y_pred_hs)

print(f"HS Accuracy: {hs_accuracy:.4f}")

In [ ]:
# Save HS model
hs_cb_model.save_model('data\\hs_cb_model.cbm')

In [ ]:
# Initialize CatBoost Regressor
rul_cb_model = CatBoostRegressor(
    depth=9,
    learning_rate=0.1,
    iterations=5000,
    loss_function='RMSE',
    eval_metric='MAE',
    verbose=1,
    random_state=42,
    early_stopping_rounds=50
)
# Train RUL model
rul_cb_model.fit(x_train_rul, y_rul_train, eval_set=(x_validation_rul, y_rul_validation))

# Evaluate Model
y_pred_best_rul = rul_cb_model.predict(x_validation_rul)
rul_mae = mean_absolute_error(y_rul_validation, y_pred_best_rul)

print(f"RUL CatBoost MAE: {rul_mae:.4f}")

In [10]:
# Save the RUL model
rul_cb_model.save_model('data\\rul_cb_model.cbm')